In [1]:
try:
    import google.colab
    google.colab.drive.mount('/content/drive/')
    %cd ../content/drive/Othercomputers/Laptop/Flood_Detection
except:
    pass

In [6]:
!pip install albumentations

  Using cached albumentations-1.3.0-py3-none-any.whl (123 kB)
  Using cached opencv_python_headless-4.7.0.72-cp37-abi3-win_amd64.whl (38.1 MB)
  Using cached qudida-0.0.4-py3-none-any.whl (3.5 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import sys; sys.path.append('../../')
from sklearn import svm
import numpy as np
from sklearn.metrics import classification_report
from sklearn import metrics
from mlpath import mlquest as mlq
from DataPreparation.DataPreparation import read_data
from FeatureExtraction.Glcm.Glcm import apply_glcm
from sklearn.metrics import classification_report
mlq.start_quest("Glcm-SVM", table_dest="../../", log_defs=True)

### Reading and Preprocessing

In [11]:
x_train_d, x_val_d, y_train_d, y_val_d = mlq.l(read_data)(saved=False)

100%|██████████| 461/461 [01:12<00:00,  6.34it/s]


x_train shape:  (737, 256, 256, 3)
x_val shape:  (185, 256, 256, 3)


### Feature Extraction: Histogram Features

In [19]:
x_train_f = mlq.l(apply_glcm)(x_train_d, distance=1, positions=[(1,1)])
x_val_f = mlq.l(apply_glcm)(x_val_d, distance=1, positions=[(1,1)])
x_train_f = x_train_f.reshape(x_train_f.shape[0], -1)
x_val_f = x_val_f.reshape(x_val_f.shape[0], -1)

(185, 16)


### Initiating the Model

In [13]:
model = mlq.l(svm.SVC)(kernel='rbf', C=7)

### Training the Model

In [20]:
model.fit(x_train_f, y_train_d)

SVC(C=7)

### Validating the Model

In [21]:
F1 = metrics.f1_score(y_val_d, model.predict(x_val_f), average='macro').item()
Accuracy = metrics.accuracy_score(y_val_d, model.predict(x_val_f)).item()
mlq.log_metrics(Accuracy, F1)
report = classification_report(y_val_d, model.predict(x_val_f))
print(report)

              precision    recall  f1-score   support

           0       0.68      0.62      0.65        87
           1       0.69      0.74      0.72        98

    accuracy                           0.69       185
   macro avg       0.69      0.68      0.68       185
weighted avg       0.69      0.69      0.69       185



### Tracking

In [22]:
mlq.end_quest()
mlq.show_logs(quest_name='Glcm-SVM', table_dest='../../', last_k=5)